# Part 8: Deploying Agents to Vertex AI Agent Engine

In this notebook, you'll learn how to deploy your multi-agent system to Google Cloud's Vertex AI Agent Engine for production use.

## What You'll Learn
- How to prepare agents for cloud deployment
- Deploy agents to Vertex AI Agent Engine
- Test deployed agents using RemoteRunner
- Monitor and manage deployed agents
- Handle agent updates and versioning

## Prerequisites
- Google Cloud Project with billing enabled
- Vertex AI API enabled
- Proper IAM permissions (Vertex AI User, Storage Admin)
- Completed Parts 1-7 of this tutorial

## Step 1: Install Required Packages

In [ ]:
!pip install -q google-cloud-aiplatform[adk,agent_engines] python-dotenv

## Step 2: Set Up Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

# Get GCP configuration
PROJECT_ID = os.environ.get('GOOGLE_CLOUD_PROJECT')
LOCATION = os.environ.get('GOOGLE_CLOUD_LOCATION', 'us-central1')
STAGING_BUCKET = os.environ.get('STAGING_BUCKET')

print(f"Project ID: {PROJECT_ID}")
print(f"Location: {LOCATION}")
print(f"Staging Bucket: {STAGING_BUCKET}")

if not PROJECT_ID:
    raise ValueError("GOOGLE_CLOUD_PROJECT environment variable is required")

## Step 3: Authenticate with Google Cloud

In [ ]:
# Authenticate with Google Cloud
# In Colab, use google.colab.auth
# In local environment, use gcloud auth application-default login

try:
    from google.colab import auth
    auth.authenticate_user()
    print("Authenticated with Google Cloud (Colab)")
except ImportError:
    # Running locally
    print("Running locally. Make sure you've run 'gcloud auth application-default login'")
    print("Or set GOOGLE_APPLICATION_CREDENTIALS environment variable")

## Step 4: Initialize Vertex AI

In [ ]:
import vertexai
from vertexai import agent_engines

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

print(f"Vertex AI initialized for project: {PROJECT_ID}")
print(f"Region: {LOCATION}")
print(f"Staging bucket: {STAGING_BUCKET}")

## Step 5: Verify Required APIs are Enabled

In [ ]:
# Check if required APIs are enabled
!gcloud services list --enabled --filter="name:(aiplatform.googleapis.com OR storage-api.googleapis.com)" --project={PROJECT_ID}

## Step 6: Prepare Your Agent for Deployment

Load your content creation coordinator agent that we built in previous parts.

In [ ]:
import sys
sys.path.append('../backend')

from agents.content_creation_coordinator import create_content_creation_coordinator

# Create the agent
agent = create_content_creation_coordinator()

print("Agent created successfully")
print(f"Agent name: {agent.name}")
print(f"Agent description: {agent.description}")

## Step 7: Deploy Agent to Vertex AI Agent Engine

This step will deploy your agent to Google Cloud and make it accessible via API.

In [ ]:
from google.adk.agents import Agent

# Deploy the agent
print("Deploying agent to Vertex AI Agent Engine...")
print("This may take 5-10 minutes...")

deployed_agent = agent.deploy(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

print("\n✅ Agent deployed successfully!")
print(f"\nAgent Resource Name: {deployed_agent.resource_name}")
print("\nSave this resource name - you'll need it to connect to your agent!")

# Save to environment variable file
with open('../.env', 'a') as f:
    f.write(f"\nAGENT_RESOURCE_NAME={deployed_agent.resource_name}\n")

print("\n✅ Resource name saved to .env file")

## Step 8: Test Your Deployed Agent

Now let's test the deployed agent using the RemoteRunner.

In [ ]:
# Get the deployed agent
# If you just deployed, use the deployed_agent from above
# Otherwise, get it by resource name:

# Option 1: Use just-deployed agent
remote_agent = deployed_agent

# Option 2: Get existing agent by resource name
# AGENT_RESOURCE_NAME = "projects/YOUR_PROJECT/locations/us-central1/reasoningEngines/YOUR_ID"
# remote_agent = agent_engines.get(AGENT_RESOURCE_NAME)

print(f"Connected to agent: {remote_agent.resource_name}")

## Step 9: Create a Session and Test Content Creation

In [ ]:
import asyncio

# Create a session
user_id = "test_user_001"
session = await remote_agent.async_create_session(user_id=user_id)
session_id = session['id']

print(f"Session created: {session_id}")

# Test query
query = """Create a complete content package for:
- Topic: Sustainable Fashion
- Target Audience: Environmentally conscious millennials
- Tone: Inspirational and Educational
- Keywords: eco-friendly, sustainable, ethical fashion
"""

print("\nSending query to agent...\n")

# Stream the response
async for event in remote_agent.async_stream_query(
    user_id=user_id,
    session_id=session_id,
    message=query
):
    if isinstance(event, dict):
        content = event.get("content", event.get("parts", {}))
        
        if isinstance(content, dict):
            parts = content.get("parts", [])
            for part in parts:
                if isinstance(part, dict) and part.get("text"):
                    print(part["text"], end="", flush=True)
        
        if "text" in event:
            print(event["text"], end="", flush=True)

print("\n\n✅ Content creation completed!")

## Step 10: List All Deployed Agents

In [ ]:
# List all agents in your project
!gcloud ai reasoning-engines list --location={LOCATION} --project={PROJECT_ID}

## Step 11: Monitor Agent Performance

View logs and metrics for your deployed agent.

In [ ]:
# View recent logs
print("Recent agent logs:")
!gcloud logging read "resource.type=aiplatform.googleapis.com/ReasoningEngine" \
    --limit=10 \
    --project={PROJECT_ID} \
    --format="table(timestamp, severity, textPayload)"

## Step 12: Update Your Agent

When you make changes to your agent, you can update the deployed version.

In [ ]:
# After making changes to your agent code:
# 1. Recreate the agent with your changes
updated_agent = create_content_creation_coordinator()

# 2. Deploy the updated version (this will update the existing deployment)
print("Updating deployed agent...")
updated_deployment = updated_agent.deploy(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

print("\n✅ Agent updated successfully!")
print(f"Resource name: {updated_deployment.resource_name}")

## Step 13: Delete Agent (Cleanup)

When you're done testing or want to remove the agent, you can delete it.

In [ ]:
# WARNING: This will permanently delete your deployed agent!
# Uncomment the following lines to delete:

# AGENT_ID = "YOUR_AGENT_ID"  # Extract from resource name
# !gcloud ai reasoning-engines delete {AGENT_ID} \
#     --location={LOCATION} \
#     --project={PROJECT_ID} \
#     --quiet

print("To delete an agent, uncomment the code above and set the AGENT_ID")

## Step 14: Production Deployment with FastAPI

For production use, deploy the FastAPI backend that connects to your Agent Engine.

### Deploy to Cloud Run

```bash
# From the project root directory:
cd deployment
./deploy-combined.sh
```

This will:
1. Build a Docker image with both frontend and backend
2. Push it to Google Artifact Registry
3. Deploy to Cloud Run
4. Set environment variables including AGENT_RESOURCE_NAME

### Environment Variables for Production

Make sure these are set in your Cloud Run service:
- `AGENT_RESOURCE_NAME`: Your agent's resource name from Step 7
- `GOOGLE_CLOUD_PROJECT`: Your GCP project ID
- `GOOGLE_CLOUD_LOCATION`: Region (e.g., us-central1)

### Testing Production Deployment

```bash
# Get your Cloud Run URL
gcloud run services describe content-studio-api \
    --region=us-central1 \
    --format='value(status.url)'

# Test the API
curl -X POST "YOUR_CLOUD_RUN_URL/api/create-content" \
    -H "Content-Type: application/json" \
    -d '{
        "topic": "AI in Healthcare",
        "target_audience": "Healthcare professionals",
        "tone": "Professional and Informative",
        "keywords": "AI, healthcare, diagnosis"
    }'
```

## Summary

In this notebook, you learned how to:

✅ Set up your GCP environment for Agent Engine deployment  
✅ Deploy an agent to Vertex AI Agent Engine  
✅ Test deployed agents using RemoteRunner  
✅ Monitor agent performance with Cloud Logging  
✅ Update and manage deployed agents  
✅ Deploy a production API with Cloud Run  

## Next Steps

1. **Scale Your Deployment**: Configure autoscaling in Cloud Run
2. **Add Authentication**: Implement API keys or OAuth for your API
3. **Monitor Costs**: Set up billing alerts and budgets
4. **Implement Caching**: Add Redis for session management
5. **Set Up CI/CD**: Automate deployments with Cloud Build

## Resources

- [Vertex AI Agent Engine Documentation](https://cloud.google.com/vertex-ai/docs/agent-engine)
- [Google ADK Documentation](https://github.com/google/adk)
- [Cloud Run Documentation](https://cloud.google.com/run/docs)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)

## Troubleshooting

### Common Issues

1. **Permission Denied**: Make sure you have Vertex AI User and Storage Admin roles
2. **API Not Enabled**: Run `gcloud services enable aiplatform.googleapis.com`
3. **Staging Bucket Error**: Create bucket with `gsutil mb gs://YOUR_BUCKET_NAME`
4. **Import Error**: Ensure you have the latest version of google-cloud-aiplatform

### Getting Help

- Check the [GitHub Issues](https://github.com/google/adk/issues)
- Review [Vertex AI samples](https://github.com/GoogleCloudPlatform/vertex-ai-samples)
- Ask questions on [Stack Overflow](https://stackoverflow.com/questions/tagged/vertex-ai)